In [1]:
from aut import *
from pyspark.sql.functions import asc, countDistinct, first

In [2]:
images = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities") \
  .images() \
  .select(remove_prefix_www(extract_domain("url")).alias("domain"), "url", "md5")

In [3]:
images.show(5, True)

+-------------+--------------------+--------------------+
|       domain|                 url|                 md5|
+-------------+--------------------+--------------------+
|geocities.com|http://geocities....|17827882f7bf42860...|
|geocities.com|http://geocities....|899bc6e3309b0fc78...|
|geocities.com|http://geocities....|955f6c342ffed6823...|
|geocities.com|http://geocities....|47718718ddfd7d43a...|
|geocities.com|http://geocities....|8ada65828daff258e...|
+-------------+--------------------+--------------------+
only showing top 5 rows



In [4]:
links = images.groupBy("md5") \
              .count() \
              .where(countDistinct("domain")>=2)

In [5]:
result = images.join(links, "md5") \
               .groupBy("domain", "md5") \
               .agg(first("url").alias("image_url")) \
               .orderBy(asc("md5"))

In [6]:
result.show(10, False)

+---------------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|domain                           |md5                             |image_url                                                                                                                                                     |
+---------------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cmsimg.marshfieldnewsherald.com  |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.marshfieldnewsherald.com/apps/pbcsi.dll/bilde?Avis=U0&Dato=20091026&Kategori=CWS01&Lopenr=910260803&Ref=PH&NewTbl=1&Itemnr=1&maxW=130&Border=0  |
|cmsimg.stevenspointjournal.com   |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.steven